In [9]:
from __future__ import print_function

import numpy as np
import time
import argparse
from pathlib import Path

import cv2
import datetime
import os
#import ffmpeg

import torch

import torch.nn as nn

from models.experimental import Ensemble
from models.common import Conv, DWConv
from utils.general import non_max_suppression, apply_classifier


In [7]:
def load_class_names(namesfile):
    class_names = []
    with open(namesfile, 'r') as fp:
        lines = fp.readlines()
    for line in lines:
        line = line.rstrip()
        class_names.append(line)
    return class_names

In [3]:
class YOLOv7_Main():
    def __init__(self, args, weightfile):
        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            self.device = 'cuda'
        else:
            self.device = 'cpu'

        self.model = Ensemble()
        ckpt = torch.load(weightfile, map_location=self.device)
        self.model.append(ckpt['ema' if ckpt.get('ema') else 'model'].float().fuse().eval())  # FP32 model

        # Compatibility updates
        for m in self.model.modules():
            if type(m) in [nn.Hardswish, nn.LeakyReLU, nn.ReLU, nn.ReLU6, nn.SiLU]:
                m.inplace = True  # pytorch 1.7.0 compatibility
            elif type(m) is nn.Upsample:
                m.recompute_scale_factor = None  # torch 1.11.0 compatibility
            elif type(m) is Conv:
                m._non_persistent_buffers_set = set()  # pytorch 1.6.0 compatibility

        self.model = self.model.half()
        self.model.eval()

        self.class_names = load_class_names(args.labels)


    def run(self, frame, args):
        sized = cv2.resize(frame, (640, 640))
        sized = cv2.cvtColor(sized, cv2.COLOR_BGR2RGB)

        image = sized / 255.0
        image = image.transpose((2, 0, 1))
        image = torch.from_numpy(image).to(self.device).half()
        image = image.unsqueeze(0)

        with torch.no_grad():
            pred = self.model(image)[0]
            pred = non_max_suppression(pred, args.det_thr, args.iou_thres, classes=args.classes, agnostic=True)

        return pred




In [4]:
def main(args):
    yolov7_main = YOLOv7_Main(args, args.weight)
    
    img_path = Path(args.path)

    for filename in img_path.glob('*.jpg'):
        frame = cv2.imread(str(filename))
        width  = frame.shape[1]
        height = frame.shape[0]

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = yolov7_main.run(frame, args)
        results = np.asarray(results[0].cpu().detach())
        break
            


In [5]:
"""Parse input arguments."""
parser = argparse.ArgumentParser(description='SORT demo')
parser.add_argument('-path', type=str, default='./data/')
parser.add_argument('-weight', type=str, default='yolov7.pt')

# Data
parser.add_argument('-labels', dest='labels',
                    action='store', default='coco.names', type=str,
                    help='Labels for detection')
parser.add_argument("-detection-thres", dest='det_thr', type=float, default=0.5)
parser.add_argument('-iou-thres', type=float, default=0.45, help='IOU threshold for NMS')
parser.add_argument('-classes', nargs='+', type=int, default=[2, 3, 4, 5, 7, 8],  help='filter by class: -class 0, or -class 0 2 3')

args = parser.parse_args(args=[])

In [6]:
yolov7_main = YOLOv7_Main(args, args.weight)

img_path = Path(args.path)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


/home/braut/.conda/envs/yolo/lib/python3.10/site-packages/torch/_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1660136134696/work/build/aten/src/ATen/core/TensorBody.h:477.)
  return self._grad


In [7]:
for filename in img_path.glob('*.jpg'):
    print(filename)
    frame = cv2.imread(str(filename))
    frame = frame[400:800, 0:500, :]
    width  = frame.shape[1]
    height = frame.shape[0]

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = yolov7_main.run(frame, args)
    results = np.asarray(results[0].cpu().detach())
    print(results)
    if len(results) > 0:
        for result in results:
            result[0] = result[0] * width/640  ## x1
            result[1] = result[1] * height/640  ## y1
            result[2] = result[2] * width/640  ## x2
            result[3] = result[3] * height/640  ## y2
            results[:, 2:4] += results[:, 0:2] #convert to [x1,y1,w,h] to [x1,y1,x2,y2]
            dets = results
            print(dets)

    #break

data/enatwrcamtarmacC1.a1.20221001.120723.jpg


/home/braut/.conda/envs/yolo/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1660136134696/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[]
data/enatwrcamtarmacC1.a1.20221002.140341.jpg


RuntimeError: Couldn't load custom C++ ops. This can happen if your PyTorch and torchvision versions are incompatible, or if you had errors while compiling torchvision from source. For further information on the compatible versions, check https://github.com/pytorch/vision#installation for the compatibility matrix. Please check your PyTorch version with torch.__version__ and your torchvision version with torchvision.__version__ and verify if they are compatible, and if not please reinstall torchvision so that it matches your PyTorch install.

In [16]:

filename= '/home/braut/yolo-detect/data/enatwrcamtarmacC1.a1.20221001.124023.jpg'
frame = cv2.imread(str(filename))
frame = frame[400:800, 0:500, :]
width  = frame.shape[1]
height = frame.shape[0]

frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
results = yolov7_main.run(frame, args)
results = np.asarray(results[0].cpu().detach())
print(results)
if results != []:
    for result in results:
        result[0] = result[0] * width/640  ## x1
        result[1] = result[1] * height/640  ## y1
        result[2] = result[2] * width/640  ## x2
        result[3] = result[3] * height/640  ## y2
        results[:, 2:4] += results[:, 0:2] #convert to [x1,y1,w,h] to [x1,y1,x2,y2]
        dets = results
        print(dets)

#break

RuntimeError: Couldn't load custom C++ ops. This can happen if your PyTorch and torchvision versions are incompatible, or if you had errors while compiling torchvision from source. For further information on the compatible versions, check https://github.com/pytorch/vision#installation for the compatibility matrix. Please check your PyTorch version with torch.__version__ and your torchvision version with torchvision.__version__ and verify if they are compatible, and if not please reinstall torchvision so that it matches your PyTorch install.

In [10]:
torch.cuda.is_available()

False